In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

In [2]:
def load_data(dataset):
    df = pd.read_csv('{}/{}.csv'.format('../data/input',dataset))
    
    return df

## Load data

In [3]:
audios = load_data('labeled_audios')
# audios.head()

In [4]:
unl_audios = load_data('unlabeled_audios')
# unl_audios.head()

In [5]:
manual = load_data('manual_selection')
manual.head()

,FECHA,Document Title,Document Url,answer_El audio parece estar relacionado con:,answer_Más información (Opcional),answer_Palabras clave/ otros nombres/ lugares (Opcional),Nuevos Comentarios,Origen,Destino,Datos de la celda Calle,PERSONAS,LUGARES,TEMAS,columbia_words
0,5/14/2013,B-1009-2013-05-14-131007-12.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,"Pacto con Irán, Causa AMIA, Irán, Israel, Vene...","""Le dije todas las propuestas a delia y el ya ...",NaN,que fernando le digo a persico. no es de gob a...,1132384699,9.890000e+11,Arevalo,"heshman, D'elia, Cancilleria, Cuervo Larroque,...",teheran,"comercio, trigo, soja, arroz","cuervo, larroque"
1,5/20/2013,B-1009-2013-05-20-153449-10.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,"Pacto con Irán, Causa AMIA, Irán, Israel, Vene...",audio del acuerdo parlamentario con Irán- Audi...,"Luis D´Ellia- Alejandro ""Yussuf"" Khalil, escuh...",Luis d'elia y yussuf escuchan audio de una ent...,1132384699,1.154097e+09,Arevalo,"Luis d'elia, ali pagdaman, encargado de negoci...",iran,"memorandum, acuerdo parlamentario",iran
2,5/20/2013,B-1009-2013-05-20-132253-6.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,"Pacto con Irán, Causa AMIA, Irán, Israel, Vene...","""Los dólares también estaban en la embajada"". ...","Eduardo, ""el gordo""",estan dispuestos a mandar los jefes de YPF a c...,1132384699,9.890000e+11,Av. Diaz Velez,"abdul karim, sheik, daniel, el gordo, d'elia, ...",embajada,"memorandum, YPF, petroleo","abdul karim, sheik, daniel, el gordo, d'elia, ..."
3,5/20/2013,B-1009-2013-05-20-133130-8.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,"Pacto con Irán, Causa AMIA, Irán, Israel, Vene...",relaciones bilaterales y comercio petróleo x g...,De Vido Min 5:15,hay 4 o 5 candidatos a presidente de iran que ...,1132384699,9.890000e+11,Ravignani,"abdul karim, de vido, cancilleria, YPF","Caracas, Buenos Aires","memorandum, relacines comerciales, YPF, estado...","granos, petroleo, de vido, devido"
4,5/15/2013,B-1009-2013-05-15-105815-4.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,Otros temas interesantes para investigar (se h...,"""estuve hablando con funcionarios del gobierno...","oscar parrilli, funcionarios,",nos vemos en la marcha,1132384699,1.161270e+09,Ravignani,"salam, namir, carloncho, funcionarios de gobie...",NaN,funcionarios,"parrili, parrilli, oscar parrilli"


In [6]:
audios = audios[["Document Title","Document Url","answer_El audio parece estar relacionado con:","length"]]
unl_audios = unl_audios[["Document Title","Document Url","length"]]
manual = manual[["Document Title","Document Url","columbia_words"]]

## Sample labeled features

In [7]:
audios_merged = pd.merge(audios, manual, on='Document Title', how='left')
audios_merged.head()

,Document Title,Document Url_x,answer_El audio parece estar relacionado con:,length,Document Url_y,columbia_words
0,01.21299.20121127.142339.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,Otros temas no relevantes,52.740,NaN,NaN
1,B-1009-2013-07-27-002452-30.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,"Descartar: no atienden, contestador, no se ent...",27.000,NaN,NaN
2,B-1009-2013-08-01-204601-14.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,Otros temas interesantes para investigar (se h...,56.088,NaN,NaN
3,B-1009-2013-09-06-150531-22.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,Otros temas no relevantes,215.640,NaN,NaN
4,B-1009-2013-01-26-175819-4.mp3,http://s3.amazonaws.com/archivo.lanacion.com.a...,Otros temas no relevantes,117.216,NaN,NaN


In [8]:
m = audios_merged.drop('Document Url_y', 1)
m = m.rename(columns={'Document Url_x': 'Document Url'})
manual_selection = m[m['columbia_words'].notnull()]
m_rest = m.loc[~m.index.isin(manual_selection.index)]

In [ ]:
l_discard = m_rest[m_rest["answer_El audio parece estar relacionado con:"].str.startswith('Descartar:')]
l_keep = m_rest.loc[~m_rest.index.isin(l_discard.index)]

In [ ]:
l_discard_sample = l_discard.sample(n=1500)
l_keep_sample = l_keep.sample(n=(1500-len(manual_selection)))

In [ ]:
labeled_sample = pd.concat([manual_selection, l_discard_sample, l_keep_sample])
labeled_sample.head()

## Sample unlabeled features

In [ ]:
# Randomly sample 7 elements from your dataframe
unlabeled_sample = unl_audios.sample(n=3000)
unlabeled_sample.head(5)

## Export to csv files

In [ ]:
import os
export_path = '../data/output/sample/'
if not os.path.exists(export_path):
    os.makedirs(export_path)

In [ ]:
labeled_sample.to_csv('{}/labeled_audios.csv'.format(export_path), index=False)

In [ ]:
unlabeled_sample.to_csv('{}/unlabeled_audios.csv'.format(export_path), index=False)